# PROJET 1 TOPIC MODELING AVIS DES PRODUITS

## Etape 1. Prétraitement des Avis de Produits

In [ ]:
# IMPORTS
import json
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from collections import defaultdict
from collections import Counter, defaultdict
import nltk
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax

### 1.Chargement des données

In [71]:
# fichier JSONL
reviews_file = 'reviews.jsonl'

# Charger les données
with open(reviews_file, 'r') as file:
    reviews = [json.loads(line) for line in file]

# Selection des champs pertinents (title et text) et création d'une liste de documents
documents = [{'title': review['title'], 'text': review['text']} for review in reviews]

# Les 5 premiers documents
print(documents[:5])

[{'title': 'No white background! It’s clear!', 'text': 'I bought this bc I thought it had the nice white background. Turns out it’s clear & since my phone is blue it doesn’t look anything like this.  If I had known that I would have purchased something else. It works ok.'}, {'title': 'Awesome!  Great price!  Works well!', 'text': 'Perfect. How pissed am I that I recently paid $20 for 1 Fitbit cable and promptly lost the damned thing?  Extremely pissed!  I keep the spare in my medicine bag so hopefully I won’t lose it and my grandson can’t get to it and try to use it as a belt or a dog leash or any of the other nutty things he’s been using the other one for.'}, {'title': 'Worked but took an hour to install', 'text': 'Overall very happy with the end result. If you hate puzzles don’t do it. I love puzzles and it worked for me. Took a lot of concentration and attention to detail and about an hour! The YouTube video helped a ton with installing the new screen. Highly recommend using a how t

### 2.Traitement linguistique

In [72]:
# Découpage en mots des documents
def tokenize_document(document):
    return word_tokenize(document['text'])

# Tokenization des documents
for document in documents:
    document['tokens'] = tokenize_document(document)
    
# Les 5 premiers documents tokenizés
print(documents[:5])


[{'title': 'No white background! It’s clear!', 'text': 'I bought this bc I thought it had the nice white background. Turns out it’s clear & since my phone is blue it doesn’t look anything like this.  If I had known that I would have purchased something else. It works ok.', 'tokens': ['I', 'bought', 'this', 'bc', 'I', 'thought', 'it', 'had', 'the', 'nice', 'white', 'background', '.', 'Turns', 'out', 'it', '’', 's', 'clear', '&', 'since', 'my', 'phone', 'is', 'blue', 'it', 'doesn', '’', 't', 'look', 'anything', 'like', 'this', '.', 'If', 'I', 'had', 'known', 'that', 'I', 'would', 'have', 'purchased', 'something', 'else', '.', 'It', 'works', 'ok', '.']}, {'title': 'Awesome!  Great price!  Works well!', 'text': 'Perfect. How pissed am I that I recently paid $20 for 1 Fitbit cable and promptly lost the damned thing?  Extremely pissed!  I keep the spare in my medicine bag so hopefully I won’t lose it and my grandson can’t get to it and try to use it as a belt or a dog leash or any of the oth

In [73]:
# Initialisation du lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatization des tokens
for document in documents:
    document['lemmas'] = [lemmatizer.lemmatize(token) for token in document['tokens']]
    
# Les 5 premiers documents lemmatisés
print(documents[:5])

[{'title': 'No white background! It’s clear!', 'text': 'I bought this bc I thought it had the nice white background. Turns out it’s clear & since my phone is blue it doesn’t look anything like this.  If I had known that I would have purchased something else. It works ok.', 'tokens': ['I', 'bought', 'this', 'bc', 'I', 'thought', 'it', 'had', 'the', 'nice', 'white', 'background', '.', 'Turns', 'out', 'it', '’', 's', 'clear', '&', 'since', 'my', 'phone', 'is', 'blue', 'it', 'doesn', '’', 't', 'look', 'anything', 'like', 'this', '.', 'If', 'I', 'had', 'known', 'that', 'I', 'would', 'have', 'purchased', 'something', 'else', '.', 'It', 'works', 'ok', '.'], 'lemmas': ['I', 'bought', 'this', 'bc', 'I', 'thought', 'it', 'had', 'the', 'nice', 'white', 'background', '.', 'Turns', 'out', 'it', '’', 's', 'clear', '&', 'since', 'my', 'phone', 'is', 'blue', 'it', 'doesn', '’', 't', 'look', 'anything', 'like', 'this', '.', 'If', 'I', 'had', 'known', 'that', 'I', 'would', 'have', 'purchased', 'somethin

In [74]:
# Charger les stop words
stop_words = set(stopwords.words('english'))

# Charger les données
with open(reviews_file, 'r') as file:
    reviews = [json.loads(line) for line in file]

# Sélection des champs pertinents (title et text) et création d'une liste de documents
documents = [{'title': review['title'], 'text': review['text']} for review in reviews]

# Découpage en mots des documents
def tokenize_document(document):
    return word_tokenize(document['text'].lower())  # Mettre en minuscule et découper en tokens

# Initialisation du lemmatizer
lemmatizer = WordNetLemmatizer()

# Nettoyage des documents
def clean_document(document):
    tokens = tokenize_document(document)
    cleaned_tokens = [
        lemmatizer.lemmatize(token)  # Lemmatization
        for token in tokens
        if token.isalpha() and token not in stop_words  # Exclusion des chiffres et stopwords
    ]
    return cleaned_tokens

# Nettoyage des documents
for document in documents:
    document['cleaned_tokens'] = clean_document(document)

# Afficher les 5 premiers documents nettoyés
for doc in documents[:5]:
    print(f"Title: {doc['title']}")
    print(f"Cleaned Tokens: {doc['cleaned_tokens']}")
    print("-" * 50)

Title: No white background! It’s clear!
Cleaned Tokens: ['bought', 'bc', 'thought', 'nice', 'white', 'background', 'turn', 'clear', 'since', 'phone', 'blue', 'look', 'anything', 'like', 'known', 'would', 'purchased', 'something', 'else', 'work', 'ok']
--------------------------------------------------
Title: Awesome!  Great price!  Works well!
Cleaned Tokens: ['perfect', 'pissed', 'recently', 'paid', 'fitbit', 'cable', 'promptly', 'lost', 'damned', 'thing', 'extremely', 'pissed', 'keep', 'spare', 'medicine', 'bag', 'hopefully', 'lose', 'grandson', 'get', 'try', 'use', 'belt', 'dog', 'leash', 'nutty', 'thing', 'using', 'one']
--------------------------------------------------
Title: Worked but took an hour to install
Cleaned Tokens: ['overall', 'happy', 'end', 'result', 'hate', 'puzzle', 'love', 'puzzle', 'worked', 'took', 'lot', 'concentration', 'attention', 'detail', 'hour', 'youtube', 'video', 'helped', 'ton', 'installing', 'new', 'screen', 'highly', 'recommend', 'using', 'video', 'r

In [75]:
# Charger les stop words
stop_words = set(stopwords.words('english'))

# Fichier d'entrée et de sortie
input_file = 'reviews.jsonl'
output_file = 'filtered_reviews.json'

# Charger les données depuis le fichier JSONL
with open(input_file, 'r') as file:
    reviews = [json.loads(line) for line in file]

# Initialisation du lemmatizer
lemmatizer = WordNetLemmatizer()

# Fonction pour nettoyer un document
def clean_document(text):
    tokens = word_tokenize(text.lower())  # Tokenisation et mise en minuscule
    cleaned_tokens = [
        lemmatizer.lemmatize(token)  # Lemmatization
        for token in tokens
        if token.isalpha() and token not in stop_words  # Garder uniquement les mots pertinents
    ]
    return cleaned_tokens

# Traitement des avis
filtered_reviews = []
for review in reviews:
    filtered_review = {
        'title': review.get('title', ''),
        'filtered_tokens': clean_document(review.get('text', ''))
    }
    filtered_reviews.append(filtered_review)

# Sauvegarde dans un fichier JSON
with open(output_file, 'w') as file:
    json.dump(filtered_reviews, file, indent=4)

print(f"Les avis filtrés ont été sauvegardés dans {output_file}.")


Les avis filtrés ont été sauvegardés dans filtered_reviews.json.


## Etape 2. Custering non supervisé des documents pour identifier des topics et mots-clés

### 1. Génération des embeddings

In [76]:
# Charger le modèle pré-entraîné
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extraire les textes des documents
texts = [doc['text'] for doc in documents]

# Générer les embeddings pour les textes
embeddings = model.encode(texts, show_progress_bar=True)

# Afficher les embeddings des 5 premiers documents
print(embeddings[:5])

Batches: 100%|██████████| 32/32 [00:02<00:00, 15.60it/s]

[[-0.06282184  0.05036959  0.08300743 ...  0.04501068 -0.02777872
   0.08845506]
 [-0.05745867  0.06035928  0.02571245 ... -0.09513349 -0.07105947
   0.03159803]
 [-0.04276645  0.03064304  0.12047602 ...  0.07421792  0.00493353
   0.04009885]
 [-0.12291301  0.03837333  0.10437208 ... -0.06036926  0.00318576
   0.05050042]
 [-0.03441106  0.02525463  0.02802392 ... -0.07721563 -0.01886018
   0.12872452]]


In [77]:
# Paramètres
eps = 0.5  # Distance maximale pour regrouper des points
min_samples = 2  # Nombre minimum de points pour former un cluster

# Charger les données filtrées
input_file = 'filtered_reviews.json'
with open(input_file, 'r') as file:
    filtered_reviews = json.load(file)

# Préparation des données textuelles
documents = [" ".join(review['filtered_tokens']) for review in filtered_reviews]

# TF-IDF vectorisation
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

# DBSCAN Clustering
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine')
dbscan_labels = dbscan.fit_predict(X)

# Ajouter les labels DBSCAN aux documents
for i, review in enumerate(filtered_reviews):
    review['dbscan_label'] = int(dbscan_labels[i])

# Sauvegarder les résultats dans un fichier JSON
output_file = 'dbscan_clustered_reviews.json'
with open(output_file, 'w') as file:
    json.dump(filtered_reviews, file, indent=4)

print(f"Les résultats de DBSCAN ont été sauvegardés dans {output_file}.")

Les résultats de DBSCAN ont été sauvegardés dans dbscan_clustered_reviews.json.


In [78]:
# Charger les résultats DBSCAN
input_file = 'dbscan_clustered_reviews.json'
with open(input_file, 'r') as file:
    clustered_reviews = json.load(file)

# Regrouper les documents par cluster
clusters = defaultdict(list)
for review in clustered_reviews:
    cluster_label = review['dbscan_label']
    clusters[cluster_label].append(review)

# Convertir en dict normal pour la sauvegarde JSON
clusters = dict(clusters)

# Sauvegarder les résultats
output_file = 'grouped_by_cluster.json'
with open(output_file, 'w') as file:
    json.dump(clusters, file, indent=4)

print(f"Les documents regroupés par cluster ont été sauvegardés dans {output_file}.")

Les documents regroupés par cluster ont été sauvegardés dans grouped_by_cluster.json.


In [ ]:
# Assurez-vous que le tokenizer et les stop words sont disponibles
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Charger les résultats DBSCAN
input_file = 'dbscan_clustered_reviews.json'
with open(input_file, 'r') as file:
    clustered_reviews = json.load(file)

# Charger les stopwords en anglais
stop_words = set(stopwords.words('english'))

# Fonction pour nettoyer et filtrer les tokens (enlever stopwords et ponctuation)
def clean_tokens(tokens):
    return [token for token in tokens if token.isalpha() and token not in stop_words]

# Regrouper les documents par cluster
clusters = defaultdict(list)
for review in clustered_reviews:
    cluster_label = review['dbscan_label']
    clusters[cluster_label].append(review)

# Calcul des fréquences des mots pour chaque cluster
cluster_word_frequencies = {}

for cluster_label, documents in clusters.items():
    all_tokens = []
    
    for document in documents:
        # Utilisation de 'filtered_tokens' qui contient déjà les tokens filtrés
        if 'filtered_tokens' in document:
            tokens = document['filtered_tokens']  # Utilisation des tokens filtrés
            cleaned_tokens = clean_tokens(tokens)  # Nettoyage des tokens (enlever stop words et ponctuation)
            all_tokens.extend(cleaned_tokens)
        else:
            print(f"Clé 'filtered_tokens' non trouvée dans le document avec le label {cluster_label}.")
    
    # Compter les fréquences des mots pour ce cluster
    word_counts = Counter(all_tokens)
    cluster_word_frequencies[cluster_label] = word_counts

# Identification des 10 mots les plus fréquents pour chaque cluster
top_words_per_cluster = {}

for cluster_label, word_counts in cluster_word_frequencies.items():
    top_words = word_counts.most_common(10)  # Récupérer les 10 mots les plus fréquents
    top_words_per_cluster[cluster_label] = top_words

# Sauvegarder les résultats dans un fichier JSON
output_file = 'top_words_per_cluster.json'
with open(output_file, 'w') as file:
    json.dump(top_words_per_cluster, file, indent=4)

print(f"Les 10 mots les plus fréquents par cluster ont été sauvegardés dans {output_file}.")

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


Les 10 mots les plus fréquents par cluster ont été sauvegardés dans top_words_per_cluster.json.


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


## Etape 3. Analyse des sentiments des avis clients

In [84]:
with open(reviews_file, 'r') as file:
    reviews = [json.loads(line) for line in file]

# Extraire les textes et les notes
texts = [review['text'] for review in reviews]
ratings = [review['rating'] for review in reviews]

# Afficher les 5 premières notes
print(ratings[:5])

[4.0, 5.0, 5.0, 4.0, 5.0]


In [85]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Charger le modèle et le tokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)



In [86]:
from torch.utils.data import DataLoader, Dataset
import torch

# Définir un Dataset personnalisé
class ReviewsDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Charger les données dans un DataLoader
batch_size = 16  # Ajustez selon vos ressources
dataset = ReviewsDataset(texts)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Préparer les prédictions
predicted_ratings = []

# Désactiver le calcul des gradients pour économiser de la mémoire
model.eval()
with torch.no_grad():
    for batch in dataloader:
        # Tokenisation
        tokens = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
        
        # Passer les données dans le modèle
        outputs = model(**tokens)
        
        # Appliquer Softmax
        probabilities = softmax(outputs.logits, dim=-1)
        
        # Prédire la classe (1 à 5)
        predictions = torch.argmax(probabilities, dim=-1) + 1  # Ajouter 1 car les classes commencent à 0
        predicted_ratings.extend(predictions.tolist())